# Bus Tracker

Adam Beigel

24 February 2024

In [17]:
import requests
import os
import json

import pandas as pd
from dotenv import load_dotenv

### Load API keys

In [3]:
load_dotenv()
bus_api_key = os.getenv('BUS_API_KEY')
train_api_key = os.getenv('TRAIN_API_KEY')

## Make Requests

### Get Stops
Note: the API docs show direction with a space and in the url as `North%20Bound` but that was changed and now it is just one word

In [ ]:
dir_abbrev = {
    'nb': 'Northbound',
    'sb': 'Southbound',
    'eb': 'Eastbound',
    'wb': 'Westbound',
}

def get_stops(route, direction, write_file=False):
    url = 'http://ctabustracker.com/bustime/api/v2/getstops'
    params = {
        'key': bus_api_key,
        'rt': route,
        'dir': dir_abbrev[direction],
        'format': 'json'
    }
    r = requests.get(url, params=params)

    if not write_file:
        return r.json()
    else:
        with open(f'data/bus_stops/{route}{direction}.json', 'w') as outfile:
            outfile.write(json.dumps(r.json(), indent=4))

In [ ]:
get_stops(36, 'nb', write_file=False)

### Get Predictions

In [88]:
def get_predictions(stops: list):
    url = 'http://www.ctabustracker.com/bustime/api/v2/getpredictions'
    params = {
        'key': bus_api_key,
        'stpid': ','.join(str(s) for s in stops),
        'format': 'json'
    }

    r = requests.get(url, params=params)
    try:
        return r.json()['bustime-response']['prd']
    except:
        return None

In [89]:
preds = get_predictions([5960])

if preds is not None:
    print(f"The time is {preds[0]['tmstmp'][9:]}")
    print(f"{preds[0]['rt']} {preds[0]['rtdir']}: {preds[0]['stpnm']}")
    for p in preds:
        print(f"   {p['prdctdn']} min ({p['prdtm'][9:]})")
else:
    print('No buses arriving in the next 30 minutes.')

The time is 12:08
8 Northbound: Halsted & Wrightwood
   8 min (12:17)
   15 min (12:24)
   26 min (12:34)


In [90]:
home_stops = [5960, 5768, 1224, 1836, 1913]
preds = get_predictions(home_stops)
df = pd.DataFrame.from_dict(preds)
# df

In [91]:
def print_preds(df):
    print(f"  Route #{df.iloc[0]['rt']}")
    for index, p in df.iterrows():
        min = 'min ' if p['prdctdn'] != 'DUE' else ''
        print(f"    {p['prdctdn']} {min}({p['prdtm'][9:]})")

def print_stops(df):
    row = df.iloc[0]
    print(f"{row['stpnm']} {row['rtdir']}")
    df.groupby('rt').apply(print_preds)
    print()

In [87]:
print(f"The time is {df.iloc[0]['tmstmp'][9:]}\n")
df.groupby('stpid').apply(print_stops)

The time is 12:07

Fullerton & Halsted Westbound
  Route #74
    6 min (12:14)
    18 min (12:25)
    29 min (12:37)

Clark & Deming Southbound
  Route #22
    5 min (12:12)
    15 min (12:22)
    21 min (12:28)
  Route #36
    9 min (12:17)

Clark & Deming Northbound
  Route #22
    9 min (12:16)
    9 min (12:16)
  Route #36
    3 min (12:11)

Halsted & Wrightwood Southbound
  Route #8
    9 min (12:16)
    20 min (12:28)

Halsted & Wrightwood Northbound
  Route #8
    9 min (12:17)
    17 min (12:24)
    27 min (12:34)



""
